## import module

In [1]:
import tensorflow as tf

## hyper Parameter

In [2]:
EPOCHS = 10

## Network

In [3]:
class MyModel(tf.keras.Model):  # tf.keras.Model을 상속
    # 사용할 layer들을 정의
    def __init__(self):
        super().__init__()  # 상위의 tf.keras.Model의 __init__을 call
        self.flatten = tf.keras.layers.Flatten(input_shape=(28, 28))
        self.dense1 = tf.keras.layers.Dense(32, activation='relu')  # 몇개의 뉴런을 쓸 것인지 지정
        self.dense2 = tf.keras.layers.Dense(64, activation='relu')
        self.dense3 = tf.keras.layers.Dense(128, activation='relu')
        self.dense4 = tf.keras.layers.Dense(256, activation='relu')
        self.dense5 = tf.keras.layers.Dense(10, activation='softmax')  # 출력 계층
        
    # layer 간의 연결성을 정의
    def call(self, x, training=None, mask=None):
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.dense4(x)
        return self.dense5(x)

## Learning Function

In [4]:
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:  # 이 안에서의 연산에 대해 Gradient 계산할 것임을 알려주는 것 -> 중간값 저장
        # Forward 계산을 수행
        predictions = model(images)  # BatchSize x 10(Classes)
        loss = loss_object(labels, predictions)  
    gradients = tape.gradient(loss, model.trainable_variables)  # loss를 모든 trainable variables로 미분
    # optimizer에 gradient와 variable을 iteration할 수 있는 개체를 넣어준다
    optimizer.apply_gradients(zip(gradients, model.trainable_variables)) 
    
    train_loss(loss)
    train_accuracy(labels, predictions)

## Test Function

In [5]:
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    predictions = model(images)  # BatchSize x 10(Classes)
    loss = loss_object(labels, predictions)  
    
    test_loss(loss)
    test_accuracy(labels, predictions)

## load data

In [6]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(1024).batch(32)  # 한번에 1024개씩 섞음, 32 크기의 배치
test_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)

## Model Creation 

In [7]:
model = MyModel()

## Define Loss Function & Optimizer

In [8]:
print(y_train)

[5 0 4 ... 5 6 8]


-> label이 sparse로 표현되어 있음

In [9]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy() 
optimizer = tf.keras.optimizers.Adam()

## Performance Measure

In [10]:
train_loss = tf.keras.metrics.Mean(name='train_loss')  # loss_object를 통해 계산되기 때문에, 평균만 취하면 된다
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

## Learning Loop

In [ ]:
for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)
    for images, labels in test_ds:
        test_step(model, images, labels, loss_object, test_loss, test_accuracy)
    
    template = 'Epoch: {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch+1, train_loss.result(), train_accuracy.result() * 100, 
                          test_loss.result(), test_accuracy.result() * 100))
    train_loss.reset_states()
    train_accuracy.reset_states()

    test_loss.reset_states()
    test_accuracy.reset_states()
    

Epoch: 1, Loss: 0.2898961007595062, Accuracy: 91.3183364868164, Test Loss: 0.19621062278747559, Test Accuracy: 94.00833129882812
